## Data  class

In [79]:
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #score of 4 or 5
            return Sentiment.POSITIVE
        
        

## loading the dataset

In [80]:
import json

file_name = './data/sentiment/Books_small_10000.json'

#take of the differece(reviews,review)
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall'])) #reviewText(text) and overallRating(score)
        
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

## Prep data

In [81]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews,test_size=0.33,random_state=42,)

In [82]:
#arrays of text and sentiment for trainning and testing set
train_x = [x.text for x in training]
train_y = [y.sentiment for y in training]

test_x = [x.text for x in test]
test_y = [y.sentiment for y in test]

### Bags of words Vectorization 

In [83]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x) ## because we dont want to fit another model so we just transform


print(train_x[0])
print(train_x_vectors[0].toarray())

#we are going to create models for our training set (train_x_vector, train_y)



Olivia Hampton arrives at the Dunraven family home as cataloger of their extensive library. What she doesn't expect is a broken carriage wheel on the way. Nor a young girl whose mind is clearly gone, an old man in need of care himself (and doesn&#8217;t quite seem all there in Olivia&#8217;s opinion). Furthermore, Marion Dunraven, the only sane one of the bunch and the one Olivia is inexplicable drawn to, seems captive to everyone in the dusty old house. More importantly, she doesn't expect to fall in love with Dunraven's daughter Marion.Can Olivia truly believe the stories of sadness and death that surround the house, or are they all just local neighborhood rumor?Was that carriage trouble just a coincidence or a supernatural sign to stay away? If she remains, will the Castle&#8217;s dark shadows take Olivia down with them or will she and Marion long enough to declare their love?Patty G. Henderson has created an atmospheric and intriguing story in her Gothic tale. I found this to be an

 ## Classification
 
 ### Lear SVM

In [84]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

 

clf_svm.predict(test_x_vectors[0]) 

array(['POSITIVE'], dtype='<U8')

### Decision Tree

In [85]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Naive Bayes

In [86]:
from sklearn.naive_bayes import GaussianNB


clf_gnb = GaussianNB()


#A sparse matrix was passed, but dense data is required. Use .todense() to convert to a dense numpy array.
train_x_vectors_dense = train_x_vectors.todense()
test_x_vectors_dense= test_x_vectors.todense()


clf_gnb.fit(train_x_vectors_dense, train_y)

clf_gnb.predict(test_x_vectors_dense[0])

array(['POSITIVE'], dtype='<U8')

### LogisticRegression

In [87]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

 

clf_log.predict(test_x_vectors[0])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['POSITIVE'], dtype='<U8')

## Evaluation

### mean Accuracy

In [88]:
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors.todense(), test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8124242424242424
0.7654545454545455
0.6587878787878788
0.8409090909090909


### F1 score

In [78]:
from sklearn.metrics import f1_score

#[y_true, x_predict,average=None, Pos_lables]

print(f1_score(test_y, clf_svm.predict(test_x_vectors),
               average=None, 
               labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGATIVE]))

print(f1_score(test_y,
               clf_dec.predict(test_x_vectors), 
               average=None, labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGATIVE]))

print(f1_score(test_y, clf_gnb.predict(test_x_vectors_dense),
               average=None, 
               labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGATIVE]))

print(f1_score(test_y,
               clf_log.predict(test_x_vectors), 
               average=None, labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGATIVE]))

#Results arrage according to pos_labels

[0.91319444 0.21052632 0.22222222]
[0.86219081 0.12698413 0.06451613]
[0.89678511 0.08510638 0.09090909]
[0.91370558 0.12244898 0.1       ]


In [92]:
train_y.count(Sentiment.POSITIVE)

5611